In [1]:
import time

import codecs

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
import pickle
import gensim
import multiprocessing
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

In [3]:
start_time = time.time() 

text = []
mecab = Mecab()

for news_name in ['chosun_full', 'donga_full', 'hani_full', 'joongang_full', 'kh_full'] :
    with open('./Data/news/'+ news_name + '.txt', 'r') as f:
            lines = f.read()
            sentences = lines.split('.')
            for i in range(len(sentences)):
                dic = mecab.morphs(sentences[i])
                dicSize = len(dic)
                
                for idx in range(dicSize):
                    dic[idx] = dic[idx].encode('utf-8')
                    
                for idx in range(dicSize-1):
                    if(idx==dicSize):
                            break

                    else:
                        if(dic[idx]=='새' and dic[(idx+1)]=='정치') :
                            dic.remove('새')
                            dic.remove('정치')
                            dic.insert(idx,"새정치")
                            dicSize = dicSize-1
                text.append(dic)
    
    print(news_name+" finished")            

with open("preprocessedText.txt", "wb") as fp:   #Pickling
    pickle.dump(text, fp)
# with open("preprocessedText.txt", "rb") as fp:   # Unpickling
  #  loadedText = pickle.load(fp)
    
    
print("Elapsed time: %s seconds" %(time.time() - start_time))

chosun_full finished
donga_full finished
hani_full finished
joongang_full finished
kh_full finished


MemoryError: 

In [4]:
import time

start_time = time.time() 


modelCbow= Word2Vec(text, min_count=15, size = 300, sg=0, iter= 10) #sg=0 : cbow
modelSkip = Word2Vec(text, min_count=15, size = 300, sg=1, iter= 10) #sg=1 : skip gram
print("word2vec success")

vectorsCbow = modelCbow.wv
vectorsSkip = modelSkip.wv
print("model.wv assign success")

vectorsCbow.save('modelCbow.bin')
vectorsSkip.save('modelSkip.bin')
print("model saved success")


print("Elapsed time: %s seconds" %(time.time() - start_time))

word2vec success
model.wv assign success
model saved success
Elapsed time: 5484.52880979 seconds


# Loading and Result

**loading models**

In [89]:
word_vectors = KeyedVectors.load('modelSkip.bin')

**word similarity**

In [80]:
name_list = ["박근혜","오바마","김정은","아베","청와대","백악관","새누리당","최순실"]

for idx in range(len(name_list)):
    name_list[idx] = name_list[idx].encode("utf-8")

similarity_list = []
for name in name_list:
    name = name.encode("utf-8")
    w = word_vectors.most_similar(name, topn=1)
    w1 = w[0][0]
    w2 = w[0][1]
    print "[%s]'s most similar: [%s]" %(name, w1),
    print(w2)
    similarity_list.append("[%s]'s most similar: [%s]" % (name, (w)))
    


[박근혜]'s most similar: [당선인] 0.737558364868
[오바마]'s most similar: [버락] 0.92061662674
[김정은]'s most similar: [김정일] 0.658897936344
[아베]'s most similar: [신조] 0.891100943089
[청와대]'s most similar: [비서관] 0.674301207066
[백악관]'s most similar: [좌관] 0.667192935944
[새누리당]'s most similar: [새정치민주연합] 0.807204842567
[최순실]'s most similar: [강난희] 0.751282215118


**word analogy**

In [111]:
def word_analogy(w1, w2, w3):
    return word_vectors.most_similar(positive=[w1, w2], negative=[w3], topn=1)

words_list = [["서울","일본","한국"],["서울","미국","한국"],["서울","북한","한국"],
                ["박근혜","일본","한국"],["박근혜","미국","한국"],["박근혜","북한","한국"],
                ["새누리당","미국","한국"],["청와대","미국","한국"],["민주주의","북한","한국"],
                ["김무성","새정치","새누리당"],["보수","새정치","새누리당"]]

for element in words_list:
    for idx in range(len(element)):
        element[idx] = element[idx].encode("utf-8")

d = {"%s+%s-%s"%(words[0],words[1],words[2]):word_analogy(words[0],words[1],words[2])[0][0] for words in words_list}

for key in d:
    print "[%s] corresponds to [%s]" % (key, d[key])


[서울+북한-한국] corresponds to [평양]
[박근혜+북한-한국] corresponds to [당선인]
[보수+새정치-새누리당] corresponds to [진보]
[민주주의+북한-한국] corresponds to [비핵화]
[새누리당+미국-한국] corresponds to [민주당]
[박근혜+일본-한국] corresponds to [아베]
[박근혜+미국-한국] corresponds to [오바마]
[서울+일본-한국] corresponds to [도쿄]
[청와대+미국-한국] corresponds to [백악관]
[김무성+새정치-새누리당] corresponds to [김한길]
[서울+미국-한국] corresponds to [워싱턴]


In [103]:
analogy_list = []

analogy_list.append(word_vectors.most_similar(positive=['서울', '일본'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['서울', '미국'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['서울', '북한'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['박근혜', '일본'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['박근혜', '미국'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['박근혜', '북한'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['새누리당', '미국'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['새정치', '미국'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['청와대', '미국'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['민주주의', '북한'], negative=['한국'], topn=1))
analogy_list.append(word_vectors.most_similar(positive=['김무성', '새정치'], negative=['새누리당'], topn=1))
analogy_list.append(word_vectorsㅌ.most_similar(positive=['보수', '새정치'], negative=['새누리당'], topn=1))

for i in range(len(analogy_list)):
    print(analogy_list[i][0][0])

도쿄
워싱턴
평양
아베
오바마
당선인
민주당
민주당
백악관
비핵화
김한길
진보
